In [5]:
import numpy as np
import pandas as pd
import torch
import os
from tqdm import tqdm

import plotly.express as px
import plotly.graph_objects as go

#### What is the close-form solution for 1-d least squares linear regression?
**(Problems 2.1, 2.2)**

Let $\phi = ax + b$ be our regression with parameters $a$ and $b$. 

Then, the loss for a datset $\mathbb{X} = \{x_i, y_i\}$ is:

\begin{align*}
L(\phi, \mathbb{X}) & = \sum_{(x_i, y_i) \in \mathbb{X}} (y_i - \phi(x_i))^2 
\\ & = \sum_{(x_i, y_i) \in \mathbb{X}} (y_i - ax_i - b)^2 
\end{align*}

To find a close-form solution, we'll calculate the gradients $dL/da$ and $dL/db$ and find the $a$ and $b$ that make the gradients 0 (ie the normal thing you do when trying to find the min/max of a function). Using the chain rule, we see:

$$dL/da = \sum_{(x_i, y_i) \in \mathbb{X}} 2 \cdot (y_i - ax_i - b) \cdot -x_i =  \sum_{(x_i, y_i) \in \mathbb{X}} 2(ax_i^2 +bx_i - y_ix_i)$$
$$dL/db = \sum_{(x_i, y_i) \in \mathbb{X}} -2 \cdot (y_i - ax_i - b) $$

<!-- Setting these to 0, we see that, 

\begin{align*}
0 & = \sum_{(x_i, y_i) \in \mathbb{X}} 2(ax_i^2 +bx_i - y_ix_i) - -2 \cdot (y_i - ax_i - b) \\ & = 2(ax_i^2 + bx_i - y_ix_i + y_i - ax_i - b) 
\\ & = 2(a(x_i^2 - x_i) + b(x_i - 1) - y_i(x_i - 1))
\\ & = 2a(x_i^2 - x_i) + 2(b - y_i)(x_i - 1) 
\\ & \implies a = \sum_{(x_i, y_i) \in \mathbb{X}}(y_i - b)(x_i - 1) / (x_i^2 - x_i)
\end{align*} -->

To solve for $b$, we see that (omitting the $\sum$ and also noting that the sum on the $b$ basically repeats it $|\mathbb{X}|$ times so that's where the means come from):

$$0 = -2y_i + 2ax_i + 2b \implies b = \bar{y_i} - a\bar{x_i}$$


To solve for $a$, we see that:
$$0 = \sum ax_i^2 + bx_i - y_ix_i \implies \sum (y_i - b)x_i = a\sum x_i^2 \implies a = \frac{\sum (y_i - b)x_i}{sumx_i^2}$$

Then, we plug in for $b$, getting:

$$a \sum x_i^2 = \sum (y_i - \bar{y_i} + a\bar{x_i})x_i$$
$$a \sum x_i^2 = \sum a\bar{x_i}x_i + \sum - \bar{y_i}x_i + y_ix_i $$
$$a \sum x_i^2  - a\sum\bar{x_i}x_i = \sum x_i(y_i - \bar{y_i}) $$
$$a = \frac{\sum x_i(y_i - \bar{y_i})}{\sum x_i(x_i - \bar{x_i})} $$

Note that you can't cancel the $x_i$ because they're different in different parts of the sum

Now, we can check if our close form solution finds the correct answer

In [18]:

data = [(1, 1), (2, 2), (3, 3)]

data = [(1.5, 0), (0, -3), (20, 37)] # y = 2x - 3

random_data = [(np.random.randint(0, 100), np.random.randint(0, 100)) for _ in range(100)]

#* github copilot generated this and it also seems to work lol
def copilot_closed_form(data):
    x = sum([x[0] for x in data])
    y = sum([x[1] for x in data])
    xy = sum([x[0]*x[1] for x in data])
    x2 = sum([x[0]**2 for x in data])
    n = len(data)
    
    w1 = (n*xy - x*y)/(n*x2 - x**2)
    w0 = (y - w1*x)/n
    return w0, w1

def my_closed_form(data):
    x_mean = np.mean([x[0] for x in data])
    y_mean = np.mean([x[1] for x in data])
    
    a = sum([x[0]*(x[1] - y_mean) for x in data]) / sum([x[0]*(x[0] - x_mean) for x in data])
    b = y_mean - a*x_mean
    return b, a
    
print(np.allclose(my_closed_form(data), copilot_closed_form(data)))
print(np.allclose(my_closed_form(random_data), copilot_closed_form(random_data)))

True
True
